[번역모델(Seq2Seq) 어텐션 메커니즘]
- 특징: 디코더의 특정 time-step의 output이 인코더의 
모든 time-step의 output 중 어떤 time-step과 가장 연관이 있는가 이다.
- 이를 수식으로 나타내면

![대체 텍스트](https://miro.medium.com/max/1225/1*UrguaZqn0agrsfnsuys31Q.png)

- e_ij는 스칼라 값이며, 디코더의 특정 timestep (i-1)의 아웃풋이 인코더의 특정 timestep (j)의 아웃풋과 얼마나 유사한지 나타내는 값이다.
- get_att_score의 함수의 return 값은 scalar 값으로 위 식의 e_ij값

In [ ]:
#get_att_score 함수
def get_att_score(self, dec_output, enc_output):        # enc_outputs [batch_size, num_directions(=1) * n_hidden]
  score = self.attn(enc_output)                         # score : [batch_size, n_hidden]
  return torch.dot(dec_output.view(-1), score.view(-1)) # inner product make scalar value

  '''
  함수 a는 s_i-1(디코더의 전 time-step의 hidden state)과 인코더의 h_j(output)를 연관짓는 
  alignment model이고, 이를 나타내는 함수 수식은 다양합니다. 
  위의 코드는 a함수를 enc_output에 linear 곱한 것을 dec_output와 내적해서 구현했습니다.
  '''

'''
alpha_ij는 e_ij 벡터에 대한 Softmax를 취한 벡터로 역시 스칼라 값입니다. 
현재 디코더의 아웃풋 time-step(i)이 어떤 인코더의 아웃풋 
time-step(j)의 연관성을 실수로 나타냅니다.
''' 
def get_att_weight(self, dec_output, enc_outputs):
  # get attention weight one 'dec_output' with 'enc_outputs'
  n_step = len(enc_outputs)
  attn_scores = Variable(torch.zeros(n_step))  
  # attn_scores : [n_step]
  for i in range(n_step):
    attn_scores[i] = self.get_att_score(dec_output, enc_outputs[i])
    # Normalize scores to weights in range 0 to 1
    
   return F.softmax(attn_scores).view(1, 1, -1)


T_x는 인코더의 총 time-step 길이를 나타냅니다.

![대체 텍스트](https://miro.medium.com/max/644/1*k8wNsogSZDxMvP-3EdKrTg.png)


In [ ]:
#seq-seq-attention
for i in range(n_step):  # each time step
	# dec_output : [n_step(=1), batch_size(=1), num_directions(=1) * n_hidden]
	# hidden : [num_layers(=1) * num_directions(=1), batch_size(=1), n_hidden]
	dec_output, hidden = self.dec_cell(dec_inputs[i].unsqueeze(0), hidden)
	attn_weights = self.get_att_weight(dec_output, enc_outputs)  # attn_weights : [1, 1, n_step]
	trained_attn.append(attn_weights.squeeze().data.numpy())

	# matrix-matrix product of matrices [1,1,n_step] x [1,n_step,n_hidden] = [1,1,n_hidden]
	context = attn_weights.bmm(enc_outputs.transpose(0, 1))
	dec_output = dec_output.squeeze(0)  # dec_output : [batch_size(=1), num_directions(=1) * n_hidden]
	context = context.squeeze(1)  # [1, num_directions(=1) * n_hidden]
	model[i] = self.out(torch.cat((dec_output, context), 1))

# make model shape [n_step, n_class]
return model.transpose(0, 1).squeeze(0), trained_attn

'''
F는 enc_outputs를 나타냅니다.
alpha_i 벡터는 위 alpha_ij 값을 encoder의 모든 time-step 만큼 가지고 있는 값입니다. 
변수 attn_weights에 해당됩니다.
c_i 벡터는 encoder의 output(F 행렬)과 위의 alpha_i 벡터를 내적한 값으로 
context = attn_weights.bmm(enc_outputs.transpose(0, 1)) 부분에 해당합니다.
'''

## **Classification의 어텐션 메커니즘(bi-LSTM with Attention)**

Seq2Seq는 encoder의 output과 decoder output의 관계로 attention을 봤다면 LSTM에서는 LSTM를 거친 

모든 outputs(contextual matrix)와 LSTM의 최종 state(query)간의 Attention을 본다는 차이가 있습니다.

![대체 텍스트](https://miro.medium.com/max/1225/1*ZhDRPhr8BUijl-UFvC-iYA.png)

위의 그림은 1층 layer로 이루어진 Classification을 위한 Bi-LSTM Attention을 나타냅니다. 

Classification에서의 Attention은 번역 모델(Seq2Seq)와 다르게 LSTM hidden cell의 

마지막 Hidden State이 어떤 time에 영향을 많이 받았는지가 포인트입니다.

In [ ]:
#bi_lstm_attention_net.py
# lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
def attention_net(self, lstm_output, final_state):
    hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
    attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
    soft_attn_weights = F.softmax(attn_weights, 1)
    # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
    context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
    return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

'''
attn_output, attention = self.attention_net(output, final_hidden_state)을 통해 
지금까지의 LSTM output과 LSTM State의 마지막 상태(final_state)를 어텐션 시킴을 알 수 있습니다.

context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
를보면 soft_attn_weights(lstm_output과 hidden를 내적 한 후 Softmax)와 
lstm_output의 내적을 통해 context 벡터를 만듭니다.

context 벡터와 nn.Linear(n_hidden * 2, num_classes)를 곱해, Classification을 하는데 이용하고 
이를 통해 학습을 진행합니다. 이를 통해 어떤 단어(time-step)가 Classification 할때 
더 많이 어텐션을 주었는지 학습 할 수 있습니다.
'''
